In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

from dataloader import get_train_set_msvr, get_test_set_msvr, get_data
from msvr import kernelmatrix
from msvr import msvr

months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [2]:
data_set = 'London_2013'
path = os.path.abspath(os.path.join(os.getcwd(), '../..'))

data = get_data(path, data_set)

100%|██████████| 2002/2002 [01:05<00:00, 30.69it/s]


In [3]:
month = 6
n_clusters = 3
method = 'hierarchical/hausdorff'

path_cluster = os.path.join(path, 'result', data_set, 'clustering', 'interval', method, f'n_clusters_{n_clusters}.csv')
clusters = pd.read_csv(path_cluster, header=None)

series = data[:, (month-1)*2:month*2, :months[month-1]*24]

In [6]:
error_train = []
error_test = []

for i in range(2,n_clusters):

    index = list(clusters[month-1] == i)
    sub_series = series[index]
    sub_series = np.sum(sub_series, axis=0)
    test = sub_series[:, -168:]
    train = sub_series[:, :-168]

    scale = np.zeros(2)
    scale[0] = np.max(train)
    scale[1] = np.min(train)
    train = (train - scale[1])/(scale[0] - scale[1])
    test = (test - scale[1])/(scale[0] - scale[1])

    # recency effect
    lag = 24
    d = 1

    trainX, trainY = get_train_set_msvr(train, lag, d)
    testX, testY = get_test_set_msvr(train, test, lag, d)

    # Parameters
    ker = 'rbf'
    epsi = 0.001
    tol = 1e-10
    min_error = float('inf')

    Cs = np.arange(0.1, 4.7, 0.1)
    pars = np.arange(1, 64, 1)
    best_params = np.zeros(2)
    for i in range(len(Cs)):
        for j in range(len(pars)):

            C = Cs[i]
            par = pars[j]

            # Train
            Beta = msvr(trainX, trainY, ker, C, epsi, par, tol)

            # Predict
            K = kernelmatrix('rbf', trainX, trainX, par)
            pred = np.dot(K, Beta)

            error = np.sum((pred - trainY)**2)
            if error < min_error:
                min_error = error

                best_params[0] = C
                best_params[1] = par

                K = kernelmatrix('rbf', testX, trainX, par)
                pred_test = np.dot(K, Beta)

                best_beta = Beta
                best_pred = pred_test

    print('lag:', lag, ', error:', min_error, ', error_test:', np.sum((best_pred - testY)**2), ', C:', best_params[0], ', sigma:', best_params[1])
    
    break

/Users/kaishuai/Desktop/ProbabilisticLoadForecasting/forecasting/interval/msvr.py:42: RuntimeWarning: divide by zero encountered in true_divide
  a = 2 * C * (u - epsi) / u


lag: 24 , error: 2.407957045672262 , error_test: 0.6208092134394134 , C: 0.5 , sigma: 1.0


In [134]:
def params_search(c_start, c_end, c_step, p_start, p_end, p_step, trainX, trainY, testX):

    # Parameters
    ker = 'rbf'
    epsi = 0.001
    tol = 1e-10
    min_error = float('inf')

    Cs = np.arange(c_start, c_end, c_step)
    pars = np.arange(p_start, p_end, p_step)
    best_params = np.zeros(2)
    for i in range(len(Cs)):
        for j in range(len(pars)):

            C = Cs[i]
            par = pars[j]

            # Train
            Beta = msvr(trainX, trainY, ker, C, epsi, par, tol)

            # Predict
            K = kernelmatrix('rbf', trainX, trainX, par)
            pred = np.dot(K, Beta)

            error = np.sum((pred - trainY)**2)
            if error < min_error:
                min_error = error
                
                best_params[0] = C
                best_params[1] = par

                K = kernelmatrix('rbf', testX, trainX, par)
                pred_test = np.dot(K, Beta)
                
                best_beta = Beta
                best_pred = pred_test
    
    return best_params, best_beta, best_pred

def train_model(trainX, trainY, testX):

    best_params, best_beta, best_pred = params_search(0.1, 4.7, 0.5, 1, 64, 2, trainX, trainY, testX)
    best_params, best_beta, best_pred = params_search(np.maximum(0.1, best_params[0]-0.5), best_params[0]+0.6, 0.1, np.maximum(1, best_params[1]-2), best_params[1]+3, 1, trainX, trainY, testX)

    return best_params, best_beta, best_pred

In [135]:
# error

best_params, best_beta, best_pred = train_model(trainX, trainY, testX)
print(np.sum((best_pred - testY)**2))

1.5361708045293347


In [115]:
# example

C = 0.1
par = 2
Beta = msvr(trainX, trainY, ker, C, epsi, par, tol)
K = kernelmatrix('rbf', testX, trainX, par)
pred_test = np.dot(K, Beta)
print(np.sum((pred_test - testY)**2))

5.580226001509968
